In [8]:
from genetic_utils import *

In [2]:
data, lables = csv_data_loader("mnist-06")

In [6]:
data.shape

(7003, 784)

In [14]:
lables

array([1, 1, 1, ..., 0, 0, 0])

In [12]:

def train_VAE(data,
              latent_dim = 2,
              hidden_layer_n = [20,18,16],
              num_dims = 10,
              kl_loss_factor = 0.01,
              epochs = 100,
              batch_size = 128
              ):


    """
    Training the VAE on the data
    """

    class Sampling(layers.Layer):
        """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

        def call(self, inputs):
            z_mean, z_log_var = inputs
            batch = tf.shape(z_mean)[0]
            dim = tf.shape(z_mean)[1]
            epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
            return z_mean + tf.exp(0.5 * z_log_var) * epsilon
    
    latent_dim = latent_dim

    encoder_inputs = keras.Input(shape=(num_dims,))
    x = layers.Dense(num_dims, activation="tanh")(encoder_inputs)
    x = layers.Dense(hidden_layer_n[0], activation="tanh")(x)
    x = layers.Dense(hidden_layer_n[1], activation="tanh")(x)
    x = layers.Dense(hidden_layer_n[2], activation="tanh")(x)
    z_mean = layers.Dense(latent_dim, name="z_mean")(x)
    z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
    z = Sampling()([z_mean, z_log_var])
    encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

    latent_inputs = keras.Input(shape=(latent_dim,))
    x = layers.Dense(hidden_layer_n[2], activation="tanh")(latent_inputs)
    x = layers.Dense(hidden_layer_n[1], activation="tanh")(x)
    x = layers.Dense(hidden_layer_n[0], activation="tanh")(x)
    decoder_outputs = layers.Dense(num_dims, activation="linear")(x)
    decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

    class VAE(keras.Model):
        def __init__(self, encoder, decoder, **kwargs):
            super().__init__(**kwargs)
            self.encoder = encoder
            self.decoder = decoder
            self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
            self.reconstruction_loss_tracker = keras.metrics.Mean(
                name="reconstruction_loss"
            )
            self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

        @property
        def metrics(self):
            return [
                self.total_loss_tracker,
                self.reconstruction_loss_tracker,
                self.kl_loss_tracker,
            ]

        def train_step(self, data):
            with tf.GradientTape() as tape:
                z_mean, z_log_var, z = self.encoder(data)
                reconstruction = self.decoder(z)
                reconstruction_loss = tf.keras.losses.MeanSquaredError()(data,reconstruction)
                kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
                total_loss = reconstruction_loss + kl_loss_factor * kl_loss
        
            grads = tape.gradient(total_loss, self.trainable_weights)
            self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
            self.total_loss_tracker.update_state(total_loss)
            self.reconstruction_loss_tracker.update_state(reconstruction_loss)
            self.kl_loss_tracker.update_state(kl_loss)
            return {
                "loss": self.total_loss_tracker.result(),
                "reconstruction_loss": self.reconstruction_loss_tracker.result(),
                "kl_loss": self.kl_loss_tracker.result(),
            }
    
    creditdata = np.concatenate([data], axis=0)
    creditdata = np.expand_dims(creditdata, -1).astype("float32")

    vae = VAE(encoder, decoder)
    vae.compile(optimizer=tf.keras.optimizers.Adam())
    history = vae.fit(creditdata,epochs=epochs,batch_size=batch_size,verbose=1)

    return vae, history

In [13]:
vae, history = train_VAE(data, num_dims=784, hidden_layer_n=[512, 256, 128] )

Epoch 1/100
55/55 [==============================] - 1s 9ms/step - loss: 9311.9362 - reconstruction_loss: 8757.4277 - kl_loss: 43.7289
Epoch 2/100
55/55 [==============================] - 1s 10ms/step - loss: 7294.4964 - reconstruction_loss: 6936.1016 - kl_loss: 41.8818
Epoch 3/100
55/55 [==============================] - 1s 9ms/step - loss: 6026.2836 - reconstruction_loss: 5820.1357 - kl_loss: 28.5950
Epoch 4/100
55/55 [==============================] - 1s 9ms/step - loss: 5250.8285 - reconstruction_loss: 5131.9893 - kl_loss: 16.2516
Epoch 5/100
55/55 [==============================] - 1s 9ms/step - loss: 4771.0007 - reconstruction_loss: 4712.3242 - kl_loss: 8.0658
Epoch 6/100
55/55 [==============================] - 1s 9ms/step - loss: 4509.0457 - reconstruction_loss: 4463.6050 - kl_loss: 5.5855
Epoch 7/100
55/55 [==============================] - 1s 10ms/step - loss: 4339.4677 - reconstruction_loss: 4321.1567 - kl_loss: 4.5457
Epoch 8/100
55/55 [==============================] - 1s 

55/55 [==============================] - 1s 10ms/step - loss: 4181.5886 - reconstruction_loss: 4164.6562 - kl_loss: 11.0057
Epoch 62/100
55/55 [==============================] - 1s 10ms/step - loss: 4164.9125 - reconstruction_loss: 4164.5786 - kl_loss: 10.5447
Epoch 63/100
55/55 [==============================] - 1s 10ms/step - loss: 4171.5131 - reconstruction_loss: 4164.5620 - kl_loss: 10.4503
Epoch 64/100
55/55 [==============================] - 1s 10ms/step - loss: 4166.4309 - reconstruction_loss: 4164.9985 - kl_loss: 10.3684
Epoch 65/100
55/55 [==============================] - 1s 10ms/step - loss: 4171.4784 - reconstruction_loss: 4163.9590 - kl_loss: 10.2888
Epoch 66/100
55/55 [==============================] - 1s 10ms/step - loss: 4169.5615 - reconstruction_loss: 4164.5288 - kl_loss: 10.1918
Epoch 67/100
55/55 [==============================] - 1s 10ms/step - loss: 4176.2942 - reconstruction_loss: 4164.0737 - kl_loss: 10.0255
Epoch 68/100
55/55 [==============================] - 

In [29]:
sample = data[3, :].reshape([1, 784])

z_mean, z_log_var, z = vae.encoder(sample.astype('float32'))
reconstruction = vae.decoder(z)

reconstruction_loss = tf.keras.losses.MeanSquaredError()(sample.astype('float32'),reconstruction)
        

In [30]:
reconstruction_loss.numpy()

3634.8528

In [40]:
def vae_detect_outliers(data,
                        vae_model,
                        num_dims
                        ):

    data_mean = []

    for i in range(data.shape[0]):
            
        sample = data[i,:].reshape([1,num_dims])
        sample = sample.astype('float32')

        z_mean, z_log_var, z = vae_model.encoder(sample)
        reconstruction = vae_model.decoder(z)

        reconstruction_loss = tf.keras.losses.MeanSquaredError()(sample,reconstruction)
        
        data_mean.append(reconstruction_loss)
    
    data_mean = np.array(data_mean)
    i_mean = np.mean(data_mean)
    i_std = np.std(data_mean)
    

    threshold = i_mean + 3*i_std

    classes = []

    for i in range(data.shape[0]):
            
        sample = data[i,:].reshape([1,num_dims])
        sample = sample.astype('float32')

        z_mean, z_log_var, z = vae_model.encoder(sample)
        reconstruction = vae_model.decoder(z)

        reconstruction_loss = tf.keras.losses.MeanSquaredError()(sample,reconstruction)
        
        if reconstruction_loss > threshold:
            
            classes.append(1)
            
        else:
            
            classes.append(0)

    classes = np.array(classes)

    return classes

In [41]:
classes = vae_detect_outliers(data, vae, 784)

In [44]:
np.sum(classes)

53

In [59]:
from sklearn.metrics import confusion_matrix
import numpy as np
# sklearn.metrics.confusion_matrix(lables, 1-classes, labels=None, sample_weight=None, normalize=None)
true_labels = 1- lables
predicted_labels = classes

cm = confusion_matrix(true_labels, predicted_labels)

# Normalize the confusion matrix to get percentages
cm_percentage = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

print("Confusion Matrix (Percentage):")
print(cm_percentage)

Confusion Matrix (Percentage):
[[0.99435028 0.00564972]
 [0.86       0.14      ]]


In [60]:
cm

array([[6864,   39],
       [  86,   14]])

In [61]:
from sklearn.metrics import precision_score, recall_score, f1_score

true_labels = 1- lables
predicted_labels = classes

precision = precision_score(true_labels, predicted_labels)

# Calculate recall
recall = recall_score(true_labels, predicted_labels)

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Precision: 0.2641509433962264
Recall: 0.14
F1 Score: 0.1830065359477124


In [52]:
mat[:, 0] = mat[:, 0]/(mat[0, 0]+ mat[1, 0])
mat[:, 1] = mat[:, 1]/(mat[0, 1]+ mat[1, 1])

IndexError: too many indices for array

In [54]:
mat

array([0.26415094, 0.73584906])